In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime, timedelta
import time
import urllib.parse
import pandas as pd

contact_df = pd.read_excel("contactList.xlsx")

options = Options()

service = Service(ChromeDriverManager().install())

def send_messages(browser):

    try:
        WebDriverWait(browser, 20).until(EC.presence_of_element_located((By.ID, "side")))

        for i, row in contact_df.iterrows():
            name = row['name']
            number = row['number']
            message = f"Olá {name}, segue o link do seu dashboard atualizado: {row['text']} Favor, acessar!"
            quote = urllib.parse.quote(message)
            url = f'https://web.whatsapp.com/send?phone={number}&text={quote}'

            browser.get(url)
            WebDriverWait(browser, 60).until(EC.presence_of_element_located((By.XPATH, '//span[@data-icon="send"]'))).click()

            #print(f"{contact_df[i]}ª mesagem enviada!")

            time.sleep(20)
    except Exception as e:
        print(f"Erro durante o envio das mensagens: {e}")

def until_next_schedule(restart_times):
    now = datetime.now()
    today = now.date()
    
    for time_str in restart_times:
        scheduled_time = datetime.strptime(time_str, '%H:%M').time()
        scheduled_datetime = datetime.combine(today, scheduled_time)
        
        if scheduled_datetime > now:
            return (scheduled_datetime - now).total_seconds(), scheduled_datetime   
        
    first_time_tomorrow = datetime.combine(today + timedelta(days=1), datetime.strptime(restart_times[0], '%H:%M').time())
    return (first_time_tomorrow - now).total_seconds(), first_time_tomorrow

def main():

    browser = webdriver.Chrome(service=service, options=options)

    restart_times = ['07:30', '09:30', '12:30', '16:30', '22:00']
    
    try:
        browser.get('https://web.whatsapp.com/')

        while True:
            # today = datetime().today().weekday()

            # if today > 5:
            #     time_until_next, next_schedule_time = until_next_schedule(restart_times)

            #     if time_until_next > 0:
            #         time.sleep()
                
            #     send_messages(browser)
            # else:
            #     next_weekday = datetime.combine(datetime.now().date() + timedelta(days=(7-today)), datetime.strptime(restart_times[0], '%H:%M').time())
            #     time_until_next = (next_weekday - datetime.now()).total_seconds()
            #     time.sleep(time_until_next)

            time_until_next, next_schedule_time = until_next_schedule(restart_times) 

            if time_until_next > 0:
                time.sleep(time_until_next)   

            send_messages(browser)

            print(f"O horário do próximo envio será às {next_schedule_time.strftime('%H:%M')}")
            time_until_next, next_schedule_time = until_next_schedule(restart_times)

    finally:

        browser.quit()

if __name__ == "__main__":
    main()

O horário do próximo envio será às 14:48
O horário do próximo envio será às 14:50
